In [1]:
import pandas as pd

In [2]:
from utils.StudentLifeDataLoader import StudentLifeDataLoader

studentlife = StudentLifeDataLoader('studentlife')


In [3]:
studentlife.get_stress_data()

,resp_time,stress_level,user_id
0,2013-04-03 06:53:52,2.0,51
1,2013-03-30 22:46:23,1.0,51
2,2013-04-03 00:55:05,2.0,51
3,2013-04-04 07:31:50,1.0,51
4,2013-04-05 01:38:47,1.0,51
...,...,...,...
1660,2013-05-19 04:21:15,3.0,33
1661,2013-05-22 04:02:24,3.0,33
1662,2013-05-24 04:00:22,3.0,33
1663,2013-05-24 22:06:06,4.0,33
